<a href="https://colab.research.google.com/github/Epilef86/DNC/blob/main/Roof_M%C3%B3veis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- Respondendo aos questionamentos


1.   A Questão do Negócio:
> Qual a minha meta?

*  Encontrar imóveis que gerem lucro futuramente

> Como posso chegar?

*  Verificando os 5 imóveis que tem maior chance de ser vendido e que gerem um maior lucro

2.   O Entendimento do Negócio:
> Quais dados eu tenho disponível?

*   Dados do condado de King referentes a preço de venda, CEP, quantidade de quartos, banheiros, dentre outros que são importantes para analisar a valorização dos imóveis e sabe qual desses atributos tem maior peso na escolha do imível mais lucrativo

> Eles são relevantes para o problema?


*   Sim, todos os atributos são importantes para análise dos dados

> Eles me trazem uma solução direta?


*   Não, é necessário verificar quais dados se correlacionam e qual peso decada um na análise exploratória

3.   A Coleta de Dados

> Os dados que eu tenho fazem sentidos?


*   Sim, cada dado tem seu peso dentro do preço do imóvel

> Estão no formato que eu gostaria?

*  Não, necessário formatas os dados de acordo com cada atributo estabelecido

> O que mais eu consigo obter de informações com esses dados?

*   Encontrar a cidade de cada imóvel;
- Calcular o preço por área de imóvel;
- Calcular o lucro/perda de investimento caso o imóvel
seja vendido mais de uma vez naquele período.

4.   A Limpeza do Dados
5.   Data

*   A data veio num formato diferente do usual, portanto foi feita uma tratativa para limpar os dados
Remoção de dados outliers e valores ausentes. O dataset não apresentava valores ausentes. Já a situação de outilers, foi utilizada o percentil de 99,5% de algumas colunas para filtrar dos dados finais

6.   Exploração dos Dados

> Visualização dos Dados

*   É possível montar diversas visualizações de dados e análises, porém limitou-se a ver a média de preço por área em cada cidade, a distribuição de preço das 3 cidades com mais imóveis

> Quais as minhas métricas essenciais?

*  Para a escolha dos melhores imóveis, foi considerado o lucro de venda do imóvel dentro de 1 ano e a cidade onde ele se encontra

> Baseado nos dados, qual imóvel ou tipo de imóveis eu deveria comprar?

*   Utilizando essas métricas da etapa anterior, o ideial é escolher imóveis que apresentaram  uma revenda ao longo do período

In [34]:
#importando a biblioteca que pega a cidade utilizando o zip
!pip install pyzipcode -q


In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
from plotly.graph_objs import *
import plotly.express as px
from pyzipcode import ZipCodeDatabase


In [36]:
#Importando o dataset e verificando as 5 primeiras linhas
df = pd.read_csv('/content/kc_house_data.csv')
display(df.head())

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


In [37]:
#Analisando as 5 últimas linhas do dataset
df.tail()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
21608,263000018,20140521T000000,360000.0,3,2.50,1530,1131,3.0,0,0,...,8,1530,0,2009,0,98103,47.6993,-122.346,1530,1509
21609,6600060120,20150223T000000,400000.0,4,2.50,2310,5813,2.0,0,0,...,8,2310,0,2014,0,98146,47.5107,-122.362,1830,7200
21610,1523300141,20140623T000000,402101.0,2,0.75,1020,1350,2.0,0,0,...,7,1020,0,2009,0,98144,47.5944,-122.299,1020,2007
21611,291310100,20150116T000000,400000.0,3,2.50,1600,2388,2.0,0,0,...,8,1600,0,2004,0,98027,47.5345,-122.069,1410,1287
21612,1523300157,20141015T000000,325000.0,2,0.75,1020,1076,2.0,0,0,...,7,1020,0,2008,0,98144,47.5941,-122.299,1020,1357


In [38]:
#Analisando os dados
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21613 entries, 0 to 21612
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21613 non-null  int64  
 1   date           21613 non-null  object 
 2   price          21613 non-null  float64
 3   bedrooms       21613 non-null  int64  
 4   bathrooms      21613 non-null  float64
 5   sqft_living    21613 non-null  int64  
 6   sqft_lot       21613 non-null  int64  
 7   floors         21613 non-null  float64
 8   waterfront     21613 non-null  int64  
 9   view           21613 non-null  int64  
 10  condition      21613 non-null  int64  
 11  grade          21613 non-null  int64  
 12  sqft_above     21613 non-null  int64  
 13  sqft_basement  21613 non-null  int64  
 14  yr_built       21613 non-null  int64  
 15  yr_renovated   21613 non-null  int64  
 16  zipcode        21613 non-null  int64  
 17  lat            21613 non-null  float64
 18  long  

In [39]:
#Mudando a formatação da coluna data, o Y é maiusculo
df['date'] = pd.to_datetime(df['date'],format='%Y%m%dT000000')
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,2014-10-13,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,2014-12-09,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,2015-02-25,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,2014-12-09,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,2015-02-18,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


In [40]:
#Convertendo as colunas: sqft_living, sqft_lot, sqft_above, sqft_basement, sqft_living15 e sqft_lot15 converter para float
df = df.astype({'sqft_living':'float64','sqft_lot':'float64','sqft_above':'float64','sqft_basement':'float64','sqft_living15':'float64','sqft_lot15':'float64'})

In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21613 entries, 0 to 21612
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   id             21613 non-null  int64         
 1   date           21613 non-null  datetime64[ns]
 2   price          21613 non-null  float64       
 3   bedrooms       21613 non-null  int64         
 4   bathrooms      21613 non-null  float64       
 5   sqft_living    21613 non-null  float64       
 6   sqft_lot       21613 non-null  float64       
 7   floors         21613 non-null  float64       
 8   waterfront     21613 non-null  int64         
 9   view           21613 non-null  int64         
 10  condition      21613 non-null  int64         
 11  grade          21613 non-null  int64         
 12  sqft_above     21613 non-null  float64       
 13  sqft_basement  21613 non-null  float64       
 14  yr_built       21613 non-null  int64         
 15  yr_renovated   2161

In [42]:
#Criando uma coluna com valores nulos chamada city para poder localizar a cidade de acordo com o zipcode
df['city'] =np.nan

In [44]:
#Baixando a biblioteca pra poder identificar a cidade associada a esse zipcode
! pip install uszipcode
from uszipcode import SearchEngine
engine = SearchEngine()
for index, row in df.iterrows():
  zipcode = engine.by_zipcode(df['zipcode'][index])
  df['city'][index] = zipcode.major_city

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Exception ignored in: <function SearchEngine.__del__ at 0x7fcf9504ab90>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/uszipcode/search.py", line 196, in __del__
  File "/usr/local/lib/python3.7/dist-packages/uszipcode/search.py", line 202, in close
  File "/usr/local/lib/python3.7/dist-packages/sqlalchemy/orm/session.py", line 1811, in close
  File "/usr/local/lib/python3.7/dist-packages/sqlalchemy/orm/session.py", line 1853, in _close_impl
  File "/usr/local/lib/python3.7/dist-packages/sqlalchemy/orm/session.py", line 923, in close
  File "/usr/local/lib/python3.7/dist-packages/sqlalchemy/engine/base.py", line 2416, in close
  File "/usr/local/lib/python3.7/dist-packages/sqlalchemy/engine/base.py", line 2639, in _do_close
  File "/usr/local/lib/python3.7/dist-packages/sqlalchemy/engine/base.py", line 2625, in _close_impl
  File "/usr/local/lib/python3.7/dist-packages/sqlalchemy/engine/base.py", line 2617, in _connection_rollback_impl
  File "/usr/lo

In [45]:
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,city
0,7129300520,2014-10-13,221900.0,3,1.00,1180.0,5650.0,1.0,0,0,...,1180.0,0.0,1955,0,98178,47.5112,-122.257,1340.0,5650.0,Seattle
1,6414100192,2014-12-09,538000.0,3,2.25,2570.0,7242.0,2.0,0,0,...,2170.0,400.0,1951,1991,98125,47.7210,-122.319,1690.0,7639.0,Seattle
2,5631500400,2015-02-25,180000.0,2,1.00,770.0,10000.0,1.0,0,0,...,770.0,0.0,1933,0,98028,47.7379,-122.233,2720.0,8062.0,Kenmore
3,2487200875,2014-12-09,604000.0,4,3.00,1960.0,5000.0,1.0,0,0,...,1050.0,910.0,1965,0,98136,47.5208,-122.393,1360.0,5000.0,Seattle
4,1954400510,2015-02-18,510000.0,3,2.00,1680.0,8080.0,1.0,0,0,...,1680.0,0.0,1987,0,98074,47.6168,-122.045,1800.0,7503.0,Sammamish


In [53]:
#verificando quais imóveis foram vendidos mais de uma vez no período. 
#O [:10] é dentro de uma lista como eu faço um head(10)
df['id'].value_counts().head(10)
#Por padrão o value_counts mostra em ordem decrescente (maior para o menor) e exclui valores nulos, olhar a biblioteca

795000620     3
8651510380    2
2568300045    2
9353300600    2
4139480200    2
1954420170    2
6381500170    2
7167000040    2
9407110710    2
1000102       2
Name: id, dtype: int64

In [54]:
#Analisando o imóvel mais vendido - 3 vendas em 1 ano (2014-2015), observando que o preço aumenta a cada venda
df[df['id'] == 795000620]

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,city,free_area,sqft_free
17602,795000620,2014-09-24,115000.0,3,1.0,1080.0,6250.0,1.0,0,0,...,1950,0,98168,47.5045,-122.33,1070.0,6250.0,Seattle,5170.0,5170.0
17603,795000620,2014-12-15,124000.0,3,1.0,1080.0,6250.0,1.0,0,0,...,1950,0,98168,47.5045,-122.33,1070.0,6250.0,Seattle,5170.0,5170.0
17604,795000620,2015-03-11,157000.0,3,1.0,1080.0,6250.0,1.0,0,0,...,1950,0,98168,47.5045,-122.33,1070.0,6250.0,Seattle,5170.0,5170.0


In [55]:
#Testando pra outro imóvel que foi vendido mais de uma vez
df[df['id'] == 8651510380]

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,city,free_area,sqft_free
6789,8651510380,2014-08-21,310000.0,3,2.0,2070.0,9195.0,1.0,0,0,...,1982,0,98074,47.6491,-122.061,2080.0,9551.0,Sammamish,7125.0,7125.0
6790,8651510380,2014-12-16,539000.0,3,2.0,2070.0,9195.0,1.0,0,0,...,1982,0,98074,47.6491,-122.061,2080.0,9551.0,Sammamish,7125.0,7125.0


In [56]:
#Testando novamente pra outro imóvel que foi vendido mais de uma vez
df[df['id'] == 9407110710]

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,city,free_area,sqft_free
2493,9407110710,2014-11-07,195000.0,3,1.75,1510.0,8400.0,1.0,0,0,...,1979,0,98045,47.4476,-121.771,1500.0,10125.0,North Bend,6890.0,6890.0
2494,9407110710,2015-02-26,322000.0,3,1.75,1510.0,8400.0,1.0,0,0,...,1979,0,98045,47.4476,-121.771,1500.0,10125.0,North Bend,6890.0,6890.0


In [57]:
#Até então todos os imóveis que foram testados acima se valorizaram ao serem vendidos

In [63]:
#O Dataset apresenta a venda de imóveis e suas características na região de Maio de 2014 à Maio de 2015
#Convertendo a coluna datetime em PeriodIndex na frequência mensal e, em seguida, tirando a média usando GroupBy.mean
# calcular as médias mensais
df_avg = df.groupby(df['date'].dt.month_name(), as_index=True)['price'].mean()

fig = px.bar(df_avg, x = df_avg.index, y = df_avg, title='Average sold house price')
fig

In [ ]:
#Quanto mais tiver terreno livre, mais chances do imóvel se valorizar. Desta forma, irei calcular a área livre
#Área livre é a diferença entre a área total - área habitável
df['sqft_free'] = df['sqft_lot'] - df['sqft_living']

In [ ]:
# compute the monthly averages
df_avg = df.groupby(df['bathrooms'].dt.month_name(), as_index=True)['Price'].mean()
DateSold
December    363500.0
January     372250.0
October     350000.0
Você pode então plotar o resultado com plotly

fig = px.bar(df_avg, x = df_avg.index, y = df_avg, title='Average sold house price')
fig

In [49]:
#Estamos interessados ​​no preço por metro quadrado do espaço habitacional, do lote e dos espaços e 
#lotes circundantes. Plotamos esses quatro valores em relação ao tempo e obtemos um padrão muito
#semelhante (gráficos não fornecidos). Então, pegamos a média dessas quatro variáveis ​​e as traçamos em função do tempo.
#Chamamos essa variável de “Preço médio por pé quadrado”. Da mesma forma, calculamos o preço médio de venda por mês 
#e o plotamos em relação ao tempo.
price_sqft_avg = (df['price_sqft_living'] + df['price_sqft_lot'] + df['price_sqft_lot15']+ df['price_sqft_living15'])/4)

SyntaxError: ignored